In [1]:
import os
import re
import time
import datetime
import requests
import pandas as pd
import requests as rq
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from IPython.display import clear_output
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def foreigner(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('p')
    # print(links)
    # <span itemprop="birthPlace">

    links = soup.find_all(['span'] , attrs = { 'itemprop' : 'birthPlace'})
    for link in links:
        for c in (link.descendants):
            aa = ((c.find_next('a')).attrs)
            if 'country=US' in aa['href']:
                return False
    return True

## Should exccute the chromedriver and get the page source

In [3]:
def league(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
            
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_all_league' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
    return result

## Should exceute the chromedriver and get the page source

In [4]:
def allstar(html):
    result = []
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('div', attrs = {'class': 'leaderboard_wrapper' , 'id': 'all_leaderboard'})

    comments = soup.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        a = comment.find('div_leaderboard')
        if (a != -1):
            #print(a,comment)
            break
    ts = BeautifulSoup(comment.string, 'html.parser')
    ts = ts.find_all('div', {'id' : 'leaderboard_allstar' , 'class' : 'data_grid_box'})
    for tt in ts:
        kk = tt.table.find_all('a')
        for k in kk:
            result.append((k.text))
            
    result = list(map (lambda x : str(int(x.split()[0])-1 ) + '-' + (x.split()[0][2:])  , result))
    return result

In [5]:
def Hall(html):
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('li' , {'class'  : 'bling_special bling_hof'})
    for link in links:
        if link.text == "Hall of Fame":
            return True

In [6]:
def btn_click(wd,url):
    wd.get(url)
    script = "vjs_addClass(document.getElementById('leaderboard_allstar'),'show_all')"
    script2 = "vjs_addClass(document.getElementById('leaderboard_allleague'),'show_all')"
    btn = wd.execute_script(script)
    btn = wd.execute_script(script2)
    return(wd.page_source)

In [7]:
def get_all_info(wd,url):
    html = btn_click(wd,url)
    result = ([Hall(html) ,league(html), allstar(html), foreigner(html)])
    return (result)

In [8]:
all_data = []
df = pd.read_csv('./info/player_my_df.csv')
info = pd.read_excel('./info/info.xlsx')

dirs = !ls ./data
dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

## Crawl the table

In [9]:
def per_game_table(html):
    
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')    
    links = soup.find_all('table', { 'id' : 'per_game'} )
    
    ## Cannot find the all per game table
    if (len(links) == 0):
        return

    rows = links[0].find_all('tr')

    colname = []
    rowname = []
    for header in links[0].find_all('th', {'scope' : 'col'}):
        colname.append(header.text)
        
    for header in links[0].find_all('th', {'scope' : 'row'}):
        rowname.append(header.text)
        
    table =[]
    df = pd.DataFrame()
    
    rnum = 0
    for row in links[0].find_all('tr'):
        table = []
        columns = row.find_all('td')
        
        for column in columns:
            ex = column.text.split('-')
            try:
                if len(ex) > 1 and int(ex[0]):
                    rowname.insert(rnum, column.text)
                    continue
            except:
                pass
            table.append(column.text)
        
        ## The table is NA 
        if not table :
            continue   
            
        rnum = rnum + 1    
        df = df.append(pd.Series(table),ignore_index=True)
    
    df[colname[0]] = pd.Series(rowname)
    df.columns = colname[-len(colname) + 1 :] + colname[:-len(colname) + 1 ]
    return(df[colname])

## Load the all data processed

In [39]:
import pickle
# dirs = !ls ./data
# dirs_no = sorted(list(map (lambda x : int(x.split('.')[0]),dirs)))

with open ('./info/all_data', 'rb') as fp:
    data = pickle.load(fp)
      
info = pd.read_excel('./info/info2.xlsx')

col = ['Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

url = pd.read_csv('./info/player_my_df.csv')['5']
df = pd.DataFrame(columns=col)   
row = len(data)
ind = info.columns

for i in range(row):
    
    ## Request the web source
    response = rq.get(url[i])
    html = response.text
    per_game_table(html) 
    my_list = per_game_table(html)
    
    ## The list may be null or not ##
    if my_list is None:
        continue
    
    pp = list(my_list['Season']).index('Career')
    my_list = my_list.loc[:pp,:]

    small_df = my_list
    r , c = ((my_list.shape))
    
    ## Process empty column from the original table ##
    for j in range(len(col)):
        if col[j] not in small_df.columns:
            small_df[col[j]] = pd.Series([' ']* r)
    
    ## Process fix award ##
    ## [None, ['1988-89'], ['1989-90'], False]
    award = data[i]
    inf = list(info.iloc[i,:])
 
    if(award[0]):
        small_df['名人堂']= pd.Series([1]*r)
    else:
        small_df['名人堂']= pd.Series([0]*r)

    if(award[3]):
        small_df['外籍球員']= pd.Series([1]*r)
    else:
        small_df['外籍球員']= pd.Series([0]*r)

    ## Column initialization ##
    small_df['是否入圍全明星'] = pd.Series([0]*r)
    small_df['當年度獎項次數'] = pd.Series([0]*r)
    small_df['當年度獎項YN'] = pd.Series([0]*r)

    ## List all season ##
    all_series = list(small_df.iloc[:,0])
    for m in range(len(award[2])):
        for n in range(len(all_series)):
            if(award[2][m] == all_series[n]):
                small_df['是否入圍全明星'][n] = 1
            
    ## Count the award throught out all the career ##            

    
    for m in range(len(award[1])):
        for n in range(len(all_series)):
            if(award[1][m] == all_series[n]):
                small_df['當年度獎項次數'][n] += 1
                small_df['當年度獎項YN'][n] = 1
                
    small_df['當年度獎項次數'][r-1] = sum(list(small_df['當年度獎項次數']))

    ## Process info fix ##
    for k in range(len(inf)):
        small_df[ind[k]]= pd.Series([inf[k]] * r)
   
    ## Process series and sort the series ##
    year_play = pd.Series([0]*r)  
    all_series.pop(-1)    
    compact_id = list(set(all_series))
    compact_id = sorted(compact_id,key= lambda x : int(x.split('-')[0]))
   
    ## Process the year player play ##
    for j in range(len(all_series)):
        front = list(small_df['Tm'])[:j]
        count = 0 
        
        ## The year should minus that he didn't play
        for k in range(len(front)):
            if 'Did' in front[k]:
                count += 1
        
        year_play[j] = compact_id.index(all_series[j]) - count + 1 
    
    ### The Career play year is the same as the final ##
    year_play[j+1] = list(year_play)[-2]
    small_df['年資'] = year_play
    
    ## Combine the dataframe ##
    df = pd.concat([small_df,df],axis=0,join='outer',ignore_index=True,sort=False)
    
    ## Ouput ##
    if i % 20 == 1:
        print(i)

    if i % 200 == 1:
        clear_output()
    print(small_df)
    break 

/usr/local/lib/python3.7/site-packages/bs4/__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


     Season Age   Tm   Lg Pos     G   GS    MP   FG   FGA ...  呎  吋    磅  \
0   1985-86  22  LAL  NBA  PF    82    1  18.8  2.5   4.7 ...  6  9  220   
1   1986-87  23  LAL  NBA  PF    79   72  28.4  4.0   7.4 ...  6  9  220   
2   1987-88  24  LAL  NBA  PF    82   64  32.1  3.9   7.8 ...  6  9  220   
3   1988-89  25  LAL  NBA  PF    82   82  30.6  4.9   9.2 ...  6  9  220   
4   1989-90  26  LAL  NBA  PF    82   82  33.0  4.7   9.8 ...  6  9  220   
5   1990-91  27  LAL  NBA  PF    82   21  26.4  3.1   6.6 ...  6  9  220   
6   1991-92  28  LAL  NBA  PF    82   53  35.4  4.7   9.8 ...  6  9  220   
7   1992-93  29  LAL  NBA  PF    82   55  34.4  4.6   8.6 ...  6  9  220   
8   1993-94  30  PHO  NBA  PF    82   55  34.5  5.7  11.3 ...  6  9  220   
9   1994-95  31  PHO  NBA  SF    82   52  32.8  3.8   7.5 ...  6  9  220   
10  1995-96  32  PHO  NBA  SF    82   36  25.8  2.6   5.4 ...  6  9  220   
11  1996-97  33  TOT  NBA  PF    83   73  30.0  2.8   5.8 ...  6  9  220   
12  1996-97 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [11]:
cols = df.columns.tolist()
cols = cols[-15:] + cols[:-15]
cols = ['FirstName', 'LastName', 'From', 'To',\
        '位置', '呎', '吋', '磅', 'Colleges','出生年', '出生月', \
        '出生日', '選秀順位','名人堂' , '當年度獎項次數', '當年度獎項YN' , '是否入圍全明星','外籍球員', '年資', \
        'Season', 'Age', 'Tm', 'Lg', 'Pos', 'G', 'GS', 'MP',\
        'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',\
        'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

In [32]:
ss = df[cols]
ss.to_excel('Helloworld.xlsx')

In [36]:
r

19

In [38]:
small_df['Season']

0     1985-86
1     1986-87
2     1987-88
3     1988-89
4     1989-90
5     1990-91
6     1991-92
7     1992-93
8     1993-94
9     1994-95
10    1995-96
11    1996-97
12    1996-97
13    1996-97
14    1997-98
15    1998-99
16    1999-00
17    2000-01
18     Career
Name: Season, dtype: object

In [30]:
(ss['Season'] , ss['當年度獎項次數'])

(0        1997-98
 1        1998-99
 2        1999-00
 3        2000-01
 4        2001-02
 5        2002-03
 6        2003-04
 7        2004-05
 8        2005-06
 9        2006-07
 10       2007-08
 11       2008-09
 12       2009-10
 13       2010-11
 14        Career
 15       2003-04
 16       2004-05
 17       2005-06
 18        Career
 19       2014-15
 20       2014-15
 21       2014-15
 22        Career
 23       2017-18
 24        Career
 25       2000-01
 26       2001-02
 27       2002-03
 28       2003-04
 29       2004-05
           ...   
 33863    2016-17
 33864     Career
 33865    2000-01
 33866    2001-02
 33867    2002-03
 33868     Career
 33869    1990-91
 33870    1991-92
 33871     Career
 33872    1999-00
 33873     Career
 33874    1985-86
 33875    1986-87
 33876    1987-88
 33877    1988-89
 33878    1989-90
 33879    1990-91
 33880    1991-92
 33881    1992-93
 33882    1993-94
 33883    1994-95
 33884    1995-96
 33885    1996-97
 33886    1996-97
 33887    